## Placerank and Agentrank

In [1]:
import fred_fn as ffn
import os

In [2]:
import numpy as np

def pagerank(M, num_iterations=100, d=0.85):
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v


In [3]:
from scipy.spatial.distance import pdist, squareform

def gaussian_kernel_similarity_graph(X, sigma=1.0):
    pairwise_sq_dists = squareform(pdist(X, 'sqeuclidean'))
    K = np.exp(-pairwise_sq_dists / (2 * sigma ** 2))
    return K


In [4]:
sex_map = ffn.sex_map
race_map = ffn.race_map
p = ffn.SynthPop()
county = ["Jefferson_County_PA"]

In [5]:
ppl = p.load_people(county).rename(columns={"AGE": "age"})
ppl = ppl.merge(p.load_people_household_xref(county), on = 'ID', how = 'left').rename(columns={"PLACE": "Household", "ROLE": "Household Role"})
ppl = ppl.merge(p.load_people_school_xref(county), on = 'ID', how = 'left').rename(columns={"PLACE": "School", "ROLE": "School Role"})
ppl = ppl.merge(p.load_people_grade_xref(county), on = 'ID', how = 'left').rename(columns={"PLACE": "Grade", "ROLE": "Grade Role"})
ppl = ppl.merge(p.load_people_workplace_xref(county), on = 'ID', how = 'left').rename(columns={"PLACE": "Workplace", "ROLE": "Workplace Role"})
ppl = ppl.merge(p.load_households(county).drop('income', axis=1).rename(columns={"ID": "Household"}), on = 'Household', how = 'left').rename(columns={"LAT": "Household Latitude", "LON": "Household Longitude", "ELEV": "Household Elevation", "Block_Group": "Household Block Group"})
# location_path = os.path.join(p.path_to_pop, p.locations[county][0])
# place_files = ['workplace.txt', 'household.txt', 'block_group.txt', 'barracks.txt', 'county.txt', 
#             'grade.txt', 'school.txt', 'prison.txt', 'college_dorm.txt', 'census_tract.txt', 
#             'nursing_home.txt', 'state.txt']

In [6]:
cols = ['ID','Household', 'Workplace', 'School', 'Grade', 'Household Block Group']
ppl = ppl[-ppl['Household'].isna()]
# ppl = ppl[cols]

```mermaid
flowchart TD
a1[county df]--group by-->b1[places]
b1 -- have similar -->d1[agents]
c1[kernel functions]--calculate similarity of -->d1
c1 -- create --> e1[similarity matrix]
d1 -- create --> e1
f1 -- create -->g1[agent similarity network]
f1[network criteria] -- restrict # similar neighbors in -->e1
e1 -- create -->g1
```

In [7]:
ppl

,ID,age,sex,race,household_relationship,household_income,Household,Household Role,School,School Role,Grade,Grade Role,Workplace,Workplace Role,Household Latitude,Household Longitude,Household Elevation,Household Block Group
0,164281596,42,1,1,0.0,66000.0,1.110482e+12,1.0,NaN,NaN,NaN,NaN,5142696920000,0.0,41.135074,-78.803882,303.379042,4.206595e+11
1,164281599,41,0,1,1.0,66000.0,1.110495e+12,1.0,NaN,NaN,NaN,NaN,5142679880000,0.0,41.150754,-79.075645,221.004238,4.206595e+11
2,164281603,41,0,1,1.0,66000.0,1.110519e+12,1.0,NaN,NaN,NaN,NaN,5142677910000,0.0,41.107643,-79.093025,288.574126,4.206595e+11
3,164281604,42,1,1,0.0,66000.0,1.110534e+12,1.0,NaN,NaN,NaN,NaN,5142709080000,0.0,41.087248,-78.884425,196.602478,4.206595e+11
4,164281606,42,1,1,0.0,66000.0,1.110553e+12,1.0,NaN,NaN,NaN,NaN,5142681260000,0.0,41.009528,-79.077429,158.117196,4.206595e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44589,163904246,29,1,1,0.0,27000.0,1.110527e+12,1.0,NaN,NaN,NaN,NaN,5142657410000,0.0,41.088022,-78.873663,219.840331,4.206595e+11
44590,163904247,30,0,1,1.0,27000.0,1.110527e+12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,41.088022,-78.873663,219.840331,4.206595e+11
44591,163946027,70,0,1,0.0,8500.0,1.110535e+12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,41.107988,-78.888554,194.004726,4.206595e+11
44592,163946028,70,0,1,0.0,8500.0,1.110559e+12,1.0,NaN,NaN,NaN,NaN,NaN,NaN,40.964674,-79.198419,154.913417,4.206595e+11
